In [7]:
import json
import datetime as DT

#создаём словарь для ответа
result={}
result["result"]={}
result.get("result")["client"]={}

#считываем данные из файла
with open("DataSet.json") as DataSet_file:
     json_DataSet = json.load(DataSet_file)

# Списки уникальных договоров по каждому заёмщику
for j in json_DataSet.get("dataLiability").get("appNO"):
    dog =[]
    dogovora = (json_DataSet.get("dataLiability").get("appNO").get(j))
    dog.append(dogovora [0])
    n=len(json_DataSet.get("dataLiability").get("appNO").get(j))
    for i in range(0,n-1):
        if dogovora [i] != dogovora [i+1]:
            dog.append(dogovora[i+1])
    result.get("result").get("client")[j]= {"rules": [""],
                              "countOpenLiability": 0,
                              "countCloseLiability": 0,
                              "sumDelays": 0,
                              "statusCH": "",
                              "arrayLiability": [""]}
    result.get("result").get("client").get(j)["arrayLiability"]=dog 
    
#Колличество открытых обязательств по каждому заёмщику
for j in json_DataSet.get("dataLiability").get("dateStart"):
    kol_otk=0
    n=len(json_DataSet.get("dataLiability").get("dateEnd").get(j))
    for i in range(0,n):
        if json_DataSet.get("dataLiability").get("dateEnd").get(j)[i]== "":
            kol_otk=kol_otk+1
    result.get("result").get("client").get(j)["countOpenLiability"]=kol_otk

#Колличество закрытых обязательств по каждому заёмщику.
for j in json_DataSet.get("dataLiability").get("dateStart"):
    kol_zak=0
    n=len(json_DataSet.get("dataLiability").get("dateStart").get(j))
    for i in range(0,n):
        zak=json_DataSet.get("dataLiability").get("dateEnd").get(j)[i]
        otk=json_DataSet.get("dataLiability").get("dateStart").get(j)[i]
        if zak!= "":
            zak=DT.datetime.strptime(zak,"%d.%m.%Y").date()
            otk=DT.datetime.strptime(otk,"%d.%m.%Y").date()
        if zak>otk:
            kol_zak=kol_zak+1
    result.get("result").get("client").get(j)["countCloseLiability"]=kol_zak  

#Текущая просроченная задолженность по каждому заёмщику
for j in json_DataSet.get("dataLiability").get("sumDelay"):
    sum_dolg = 0
    for i in json_DataSet.get("dataLiability").get("sumDelay").get(j):
        sum_dolg = sum_dolg +i
    result.get("result").get("client").get(j)["sumDelays"]=sum_dolg
    
# оценка КИ. B-плохая, G - хорошая, E - отсутствует, N - средняя
for j in json_DataSet.get("dataLiability").get("sumDelay"):
    sum_dolg = 0
    kol_dog = 0
    for i in json_DataSet.get("dataLiability").get("sumDelay").get(j):
        sum_dolg = sum_dolg +i
    for i in json_DataSet.get("dataLiability").get("appNO").get(j):   
        if i !="":
            kol_dog = kol_dog+1
    if sum_dolg>1000:
        result.get("result").get("client").get(j)["statusCH"]="B"
    elif (sum_dolg>0) and (sum_dolg<1000) and (kol_dog>0):
        result.get("result").get("client").get(j)["statusCH"]="N"
    elif kol_dog == 0:
        result.get("result").get("client").get(j)["statusCH"]="E"
    elif (kol_dog>0) and (sum_dolg==0):
        result.get("result").get("client").get(j)["statusCH"]="G"

# проверка на массив правил
today = DT.date.today()
days_in_year = 365.2425
for j in json_DataSet.get("client"):
    rules=[]
    if json_DataSet.get("client").get(j).get("citizenship") != "RU":
        rules.append("rule_1")
    age=DT.datetime.strptime(json_DataSet.get("client").get(j).get('birthday'),"%d.%m.%Y").date()
    age = int((today - age).days / days_in_year)
    if age<18:
        rules.append("rule_2")
    if age>55:
        rules.append("rule_3")
    if json_DataSet.get("source").get ('egripStatus')[j] !=1:
        rules.append("rule_4")
    result.get("result").get("client").get(j)["rules"]=rules
with open('otvet.json', 'w') as f:
    json.dump(result, f, sort_keys=False, indent=4)
result=json.dumps(result, indent=6) 
print(result)

{
      "result": {
            "client": {
                  "client_1": {
                        "rules": [
                              "rule_2"
                        ],
                        "countOpenLiability": 2,
                        "countCloseLiability": 1,
                        "sumDelays": 2500,
                        "statusCH": "B",
                        "arrayLiability": [
                              "V445B4",
                              "V45V24BB56B",
                              "F452V-454"
                        ]
                  },
                  "client_2": {
                        "rules": [
                              "rule_1"
                        ],
                        "countOpenLiability": 3,
                        "countCloseLiability": 0,
                        "sumDelays": 26000,
                        "statusCH": "B",
                        "arrayLiability": [
                              "SDF-S4B44-RTB",
              